In [1]:
!pip install transformers datasets evaluate accelerate torch -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 90.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 69.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 76.3 MB/s eta 0:00:00


In [27]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, pipeline
import numpy as np
from sklearn.metrics import classification_report, accuracy_score, f1_score

In [3]:
dataset = load_dataset("dair-ai/emotion")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/127k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/129k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [5]:
dataset['train'][0:5]

{'text': ['i didnt feel humiliated',
  'i can go from feeling so hopeless to so damned hopeful just from being around someone who cares and is awake',
  'im grabbing a minute to post i feel greedy wrong',
  'i am ever feeling nostalgic about the fireplace i will know that it is still on the property',
  'i am feeling grouchy'],
 'label': [0, 0, 3, 2, 3]}

In [6]:
dataset['train'].features['label'].names

['sadness', 'joy', 'love', 'anger', 'fear', 'surprise']

In [7]:
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
model = AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=6)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
def tokenizer_fn(batch):
    return tokenizer(batch['text'], padding= 'max_length', truncation= True)

In [9]:
tokenized_dataset = dataset.map(tokenizer_fn, batched= True)

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [10]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 2000
    })
})

In [11]:
label_to_id ={}
id_to_label ={}
labels = dataset['train'].features['label'].names
for i in range(len(labels)):
    label_to_id[labels[i]] = i
for label in label_to_id:
    id_to_label[label_to_id[label]] = label

model.config.label2id = label_to_id
model.config.id2label = id_to_label

In [21]:
training_args = TrainingArguments(
    output_dir = './results',
    eval_strategy = 'epoch',
    save_strategy = 'epoch',
    logging_dir = './logs',
    per_device_train_batch_size = 16,
    per_device_eval_batch_size = 64,
    num_train_epochs = 3,
    fp16 = True,
    logging_steps = 10,
    load_best_model_at_end = True,
    metric_for_best_model = 'accuracy'
)

In [22]:
def compute_metrics(pred):
    logits, labels = pred
    preds = np.argmax(logits, axis=1)
    return {
        "accuracy" : accuracy_score(labels, preds),
        "f1" : f1_score(labels, preds, average="weighted")
    }

In [23]:
trainer = Trainer(
    model= model,
    args= training_args,
    train_dataset= tokenized_dataset['train'],
    eval_dataset= tokenized_dataset['validation'],
    compute_metrics= compute_metrics
)

In [24]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.201800,0.200668,0.933500,0.933703
2,0.069200,0.189210,0.935500,0.934778
3,0.072100,0.227920,0.940000,0.939668


TrainOutput(global_step=3000, training_loss=0.10571134444574515, metrics={'train_runtime': 673.9599, 'train_samples_per_second': 71.221, 'train_steps_per_second': 4.451, 'total_flos': 6358888710144000.0, 'train_loss': 0.10571134444574515, 'epoch': 3.0})

In [25]:
trainer.evaluate(tokenized_dataset['test'])

{'eval_loss': 0.2751978635787964,
 'eval_accuracy': 0.9245,
 'eval_f1': 0.9240171887398952,
 'eval_runtime': 8.1983,
 'eval_samples_per_second': 243.953,
 'eval_steps_per_second': 3.903,
 'epoch': 3.0}

In [26]:
trainer.save_model("emotion-classifier")
tokenizer.save_pretrained("emotion-classifier")

('emotion-classifier/tokenizer_config.json',
 'emotion-classifier/special_tokens_map.json',
 'emotion-classifier/vocab.txt',
 'emotion-classifier/added_tokens.json',
 'emotion-classifier/tokenizer.json')

In [28]:
nlp = pipeline("text-classification", model= "emotion-classifier", tokenizer= "emotion-classifier")

Device set to use cuda:0


In [29]:
print(nlp("quite a lovely day"))

[{'label': 'love', 'score': 0.9986947178840637}]


In [30]:
print(nlp("feeling excited for work"))

[{'label': 'joy', 'score': 0.9998626708984375}]


In [31]:
print(nlp("why is it monday again?"))

[{'label': 'anger', 'score': 0.9055753350257874}]


In [32]:
print(nlp("sorry for your loss"))

[{'label': 'sadness', 'score': 0.9998230338096619}]
